# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

from sqlalchemy import create_engine


In [2]:
# load data from database
engine = create_engine('sqlite:///figure_eight.db')
df = pd.read_sql_table('dataset_main',engine)
X =df['message']
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [3]:
df['message'][:5]

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
import nltk,re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def tokenize(text):
    text = re.sub("[^A-z0-9]+"," ",text)
    text = text.lower()
    words = word_tokenize(text)
    words= [w for w in words if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    lemmed = [lemmatizer.lemmatize(w).strip() for w in words]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline,FeatureUnion


In [7]:
pipeline = Pipeline([
    ('features',FeatureUnion([
        ('text_pipeline',Pipeline([
            ('vect',CountVectorizer(tokenizer=tokenize)),
            ('tfidf',TfidfTransformer(smooth_idf=False)),
            
        ])),
    ])),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
    #('clf',MultiOutputClassifier(LinearRegression()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.25,random_state=420)

In [9]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# def report(y,yhat,classes = None):
#     f1 = ['F1']
#     recall=['Recall']
#     precision = ['Precision']
#     for i in range(y.shape[1]):
#         t =(classification_report(y.iloc[:,i],yhat[:,i]))
#         f1.append(float(t[-15:-11]))#f1
#         recall.append(float(t[-25:-21]))#recall
#         precision.append(float(t[-35:-31]))#precision
#     return pd.DataFrame([precision,recall,f1],columns=['Metric']+classes).set_index('Metric')


# cfr = report(y_test,y_pred,Y.columns.tolist())
# cfr.head()

# avg_precision = cfr.iloc[0,:].sum()/cfr.shape[1]
# avg_recall = cfr.iloc[1,:].sum()/cfr.shape[1]
# avg_f1 = cfr.iloc[2,:].sum()/cfr.shape[1]

# print(f"The average Precision score is : {avg_precision}\nThe average Recall score is : {avg_recall}\nThe average F1 score is : {avg_f1}")
 

In [14]:
from sklearn.metrics import classification_report
import numpy as np

In [15]:
def report(y,yhat,classes):
    results = []
    for i in range(y.shape[1]):
        t =(classification_report(y.iloc[:,i],yhat[:,i]))
        f1=(float(t[-15:-11]))#f1
        recall=(float(t[-25:-21]))#recall
        precision=(float(t[-35:-31]))#precision
        results.append([classes[i],precision,recall,f1])
    results.append(['Average',np.average(precision),np.average(recall),np.average(f1)])
    return pd.DataFrame(results,columns=['Classes','Precision','Recall','F1']).set_index('Classes')




In [12]:
y_pred = pipeline.predict(X_test)

In [16]:
cfr = report(y_test,y_pred,Y.columns.tolist())
cfr

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Precision,Recall,F1
Classes,,,
related,0.79,0.81,0.80
request,0.88,0.89,0.87
offer,0.99,1.00,0.99
aid_related,0.75,0.75,0.75
medical_help,0.89,0.92,0.89
medical_products,0.95,0.96,0.94
search_and_rescue,0.97,0.97,0.96
security,0.97,0.98,0.97
military,0.97,0.97,0.96


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.